In [10]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import seaborn as sns

dir_in = "inputs/"

In [8]:
os.listdir(dir_in)

['biology.csv',
 'cooking.csv',
 'crypto.csv',
 'diy.csv',
 'robotics.csv',
 'test.csv',
 'travel.csv']

In [9]:
biology = pd.read_csv(dir_in + "biology.csv")
cooking = pd.read_csv(dir_in + "cooking.csv")
crypto = pd.read_csv(dir_in + "crypto.csv")
diy = pd.read_csv(dir_in + "diy.csv")
robotics = pd.read_csv(dir_in + "robotics.csv")
travel = pd.read_csv(dir_in + "travel.csv")
test = pd.read_csv(dir_in + "test.csv")

cooking.head()

,id,title,content,tags
0,1,How can I get chewy chocolate chip cookies?,<p>My chocolate chips cookies are always too c...,baking cookies texture
1,2,How should I cook bacon in an oven?,<p>I've heard of people cooking bacon in an ov...,oven cooking-time bacon
2,3,What is the difference between white and brown...,"<p>I always use brown extra large eggs, but I ...",eggs
3,4,What is the difference between baking soda and...,<p>And can I use one in place of the other in ...,substitutions please-remove-this-tag baking-so...
4,5,"In a tomato sauce recipe, how can I cut the ac...",<p>It seems that every time I make a tomato sa...,sauce pasta tomatoes italian-cuisine


## Define relevent functions for tokenizer

In [48]:
nomeaning = ['the', 'a', 'is', 'are']
def Tokenizer(stringlst):
    idx = 1
    worddic = dict()
    for phrase in stringlst:
        for word in phrase:
            if word not in worddic:
                if word not in nomeaning:
                    worddic[word] = idx
                    idx += 1
    return worddic

def phrase2seq(phrase, worddic):
    words = phrase.split()
    seq = []
    for word in words:
        if word in nomeaning:
            seq.append(0)
        else:
            seq.append(worddic[word])
    return seq

def flat(hierachy):
    l = []
    for sublst in hierachy:
        for item in sublst:
            l.append(item)
    return l

SyntaxError: invalid syntax (<ipython-input-48-1387c0dcb03f>, line 8)

## Tokenizer tags

In [37]:
biology_phrase = biology.tags.apply(lambda s: s.split()).as_matrix()
crypto_phrase = crypto.tags.apply(lambda s: s.split()).as_matrix()
diy_phrase = diy.tags.apply(lambda s: s.split()).as_matrix()
robotics_phrase = robotics.tags.apply(lambda s: s.split()).as_matrix()
travel_phrase = travel.tags.apply(lambda s: s.split()).as_matrix()
cooking_phrase = cooking.tags.apply(lambda s: s.split()).as_matrix()

s = flat([biology_phrase, crypto_phrase, diy_phrase, robotics_phrase, travel_phrase, cooking_phrase])
tagtoken = Tokenizer(s)

biology['sequence'] = biology.tags.apply(lambda s: phrase2seq(s, tagtoken))
cooking['sequence'] = cooking.tags.apply(lambda s: phrase2seq(s, tagtoken))
crypto['sequence'] = crypto.tags.apply(lambda s: phrase2seq(s, tagtoken))
diy['sequence'] = diy.tags.apply(lambda s: phrase2seq(s, tagtoken))
robotics['sequence'] = robotics.tags.apply(lambda s: phrase2seq(s, tagtoken))
travel['sequence'] = travel.tags.apply(lambda s: phrase2seq(s, tagtoken))

In [45]:
train = pd.concat((biology, cooking, crypto, diy, robotics, travel))

In [47]:
train.head(100)

,id,title,content,tags,sequence
0,1,What is the criticality of the ribosome bindin...,"<p>In prokaryotic translation, how critical fo...",ribosome binding-sites translation synthetic-b...,"[1, 2, 3, 4]"
1,2,How is RNAse contamination in RNA based experi...,<p>Does anyone have any suggestions to prevent...,rna biochemistry,"[5, 6]"
2,3,Are lymphocyte sizes clustered in two groups?,<p>Tortora writes in <em>Principles of Anatomy...,immunology cell-biology hematology,"[7, 8, 9]"
3,4,How long does antibiotic-dosed LB maintain goo...,<p>Various people in our lab will prepare a li...,cell-culture,[10]
4,5,Is exon order always preserved in splicing?,<p>Are there any cases in which the splicing m...,splicing mrna spliceosome introns exons,"[11, 12, 13, 14, 15]"
5,6,How can I avoid digesting protein-bound DNA?,<p>I'm interested in sequencing and analyzing ...,dna biochemistry molecular-biology,"[16, 6, 17]"
6,8,Under what conditions do dendritic spines form?,<p>I'm looking for resources or any informatio...,neuroscience synapses,"[18, 19]"
7,9,How should I ship plasmids?,<p>I shipped 10 µL of my vector miniprep to a ...,plasmids,[20]
8,10,What is the reason behind choosing the reporte...,<p>I noticed within example experiments in cla...,molecular-genetics gene-expression experimenta...,"[21, 22, 23]"
9,11,How many times did endosymbiosis occur?,"<p>According to the endosymbiont theory, mitoc...",evolution mitochondria chloroplasts,"[24, 25, 26]"


In [51]:
a = 'Are'
a.lower().strip()

'are'